In [11]:
from datasets import Dataset, DatasetDict

In [12]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import spacy
import datasets
import torchtext
import tqdm
import evaluate

### Dataset

In [14]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

In [15]:
def readData(lang1, lang2):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('../data/tatoeba/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    # if reverse:
    #     pairs = [list(reversed(p)) for p in pairs]
    #     input_lang = Lang(lang2)
    #     output_lang = Lang(lang1)
    # else:
    #     input_lang = Lang(lang1)
    #     output_lang = Lang(lang2)

    data = [{lang1: pair[0], lang2: pair[1]}for pair in pairs]

    return data

In [16]:
data = readData('en', 'fr')

Reading lines...


In [17]:
data[:10]

[{'en': 'go', 'fr': 'va !'},
 {'en': 'run !', 'fr': 'cours !'},
 {'en': 'run !', 'fr': 'courez !'},
 {'en': 'wow !', 'fr': 'ca alors !'},
 {'en': 'fire !', 'fr': 'au feu !'},
 {'en': 'help !', 'fr': 'a l aide !'},
 {'en': 'jump', 'fr': 'saute'},
 {'en': 'stop !', 'fr': 'ca suffit !'},
 {'en': 'stop !', 'fr': 'stop !'},
 {'en': 'stop !', 'fr': 'arrete toi !'}]

In [18]:
random.seed(42)
random.shuffle(data)

In [19]:
# splittng data
n = len(data)
n_train = int(0.8*n)
n_val = int(0.9*n)

train_data = data[:n_train]
val_data = data[n_train:n_val]
test_data = data[n_val:]

# convert to Huggingface dataset
train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)
test_dataset = Dataset.from_list(test_data)

# Create the DatasetDict
dataset_dict = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})

In [20]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['en', 'fr'],
        num_rows: 108673
    })
    validation: Dataset({
        features: ['en', 'fr'],
        num_rows: 13584
    })
    test: Dataset({
        features: ['en', 'fr'],
        num_rows: 13585
    })
})

In [21]:
train_dataset[0]

{'en': 'i ll leave it up to you', 'fr': 'je vous laisse la decision'}

In [22]:
en_nlp = spacy.load("en_core_web_sm")
fr_nlp = spacy.load("fr_core_news_sm")

In [23]:
string = "What a lovely day it is today!"

[token.text for token in en_nlp.tokenizer(string)]

['What', 'a', 'lovely', 'day', 'it', 'is', 'today', '!']

In [24]:
def tokenize_example(example, en_nlp, fr_nlp, max_length, lower, sos_token, eos_token):
    en_tokens = [token.text for token in en_nlp.tokenizer(example["en"])][:max_length]
    fr_tokens = [token.text for token in fr_nlp.tokenizer(example["fr"])][:max_length]
    if lower:
        en_tokens = [token.lower() for token in en_tokens]
        fr_tokens = [token.lower() for token in fr_tokens]
    en_tokens = [sos_token] + en_tokens + [eos_token]
    fr_tokens = [sos_token] + fr_tokens + [eos_token]
    return {"en_tokens": en_tokens, "fr_tokens": fr_tokens}

In [25]:
max_length = 1_000
lower = True
sos_token = "<sos>"
eos_token = "<eos>"

fn_kwargs = {
    "en_nlp": en_nlp,
    "fr_nlp": fr_nlp,
    "max_length": max_length,
    "lower": lower,
    "sos_token": sos_token,
    "eos_token": eos_token,
}

train_dataset = train_dataset.map(tokenize_example, fn_kwargs=fn_kwargs)
val_dataset = val_dataset.map(tokenize_example, fn_kwargs=fn_kwargs)
test_dataset = test_dataset.map(tokenize_example, fn_kwargs=fn_kwargs)

Map:   0%|          | 0/108673 [00:00<?, ? examples/s]

Map:   0%|          | 0/13584 [00:00<?, ? examples/s]

Map:   0%|          | 0/13585 [00:00<?, ? examples/s]

In [26]:
train_dataset[0]

{'en': 'i ll leave it up to you',
 'fr': 'je vous laisse la decision',
 'en_tokens': ['<sos>', 'i', 'll', 'leave', 'it', 'up', 'to', 'you', '<eos>'],
 'fr_tokens': ['<sos>', 'je', 'vous', 'laisse', 'la', 'decision', '<eos>']}

In [27]:
from torchtext.vocab import build_vocab_from_iterator
min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]

en_vocab = build_vocab_from_iterator(
    train_dataset["en_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

fr_vocab = build_vocab_from_iterator(
    train_dataset["fr_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

/opt/anaconda3/envs/myenv/lib/python3.12/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/opt/anaconda3/envs/myenv/lib/python3.12/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [28]:
en_vocab.get_itos()[:10]

['<unk>', '<pad>', '<sos>', '<eos>', 'i', 'you', 'to', 'the', '?', 'a']

In [29]:
fr_vocab.get_itos()[:10]

['<unk>', '<pad>', '<sos>', '<eos>', 'je', 'a', 'de', 'pas', '?', 'vous']

In [30]:
en_vocab['the']

7

In [31]:
len(en_vocab), len(fr_vocab)

(8146, 12220)

In [32]:
assert en_vocab[unk_token] == fr_vocab[unk_token]
assert en_vocab[pad_token] == fr_vocab[pad_token]

unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

In [33]:
en_vocab['The']

RuntimeError: Token The not found and default index is not set

In [34]:
en_vocab.set_default_index(unk_index)
fr_vocab.set_default_index(unk_index)

In [35]:
en_vocab['The']

0

In [36]:
en_vocab.get_itos()[0]

'<unk>'

In [37]:
tokens = ["i", "love", "watching", "crime", "shows"]
en_vocab.lookup_indices(tokens)

[4, 133, 530, 1039, 3345]

In [38]:
en_vocab.lookup_tokens(en_vocab.lookup_indices(tokens))

['i', 'love', 'watching', 'crime', 'shows']

In [39]:
def numericalize_example(example, en_vocab, fr_vocab):
    en_ids = en_vocab.lookup_indices(example["en_tokens"])
    fr_ids = fr_vocab.lookup_indices(example["fr_tokens"])
    return {"en_ids": en_ids, "fr_ids": fr_ids}

In [40]:
fn_kwargs = {"en_vocab": en_vocab, "fr_vocab": fr_vocab}

train_dataset = train_dataset.map(numericalize_example, fn_kwargs=fn_kwargs)
val_dataset = val_dataset.map(numericalize_example, fn_kwargs=fn_kwargs)
test_dataset = test_dataset.map(numericalize_example, fn_kwargs=fn_kwargs)

Map:   0%|          | 0/108673 [00:00<?, ? examples/s]

Map:   0%|          | 0/13584 [00:00<?, ? examples/s]

Map:   0%|          | 0/13585 [00:00<?, ? examples/s]

In [41]:
train_dataset[0]

{'en': 'i ll leave it up to you',
 'fr': 'je vous laisse la decision',
 'en_tokens': ['<sos>', 'i', 'll', 'leave', 'it', 'up', 'to', 'you', '<eos>'],
 'fr_tokens': ['<sos>', 'je', 'vous', 'laisse', 'la', 'decision', '<eos>'],
 'en_ids': [2, 4, 52, 163, 12, 63, 6, 5, 3],
 'fr_ids': [2, 4, 9, 192, 13, 701, 3]}

In [42]:
data_type = "torch"
format_columns = ["en_ids", "fr_ids"]

train_dataset = train_dataset.with_format(
    type=data_type, columns=format_columns, output_all_columns=True
)

val_dataset = val_dataset.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

test_dataset = test_dataset.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

In [43]:
train_dataset[0]

{'en_ids': tensor([  2,   4,  52, 163,  12,  63,   6,   5,   3]),
 'fr_ids': tensor([  2,   4,   9, 192,  13, 701,   3]),
 'en': 'i ll leave it up to you',
 'fr': 'je vous laisse la decision',
 'en_tokens': ['<sos>', 'i', 'll', 'leave', 'it', 'up', 'to', 'you', '<eos>'],
 'fr_tokens': ['<sos>', 'je', 'vous', 'laisse', 'la', 'decision', '<eos>']}

In [44]:
train_dataset[0]

{'en_ids': tensor([  2,   4,  52, 163,  12,  63,   6,   5,   3]),
 'fr_ids': tensor([  2,   4,   9, 192,  13, 701,   3]),
 'en': 'i ll leave it up to you',
 'fr': 'je vous laisse la decision',
 'en_tokens': ['<sos>', 'i', 'll', 'leave', 'it', 'up', 'to', 'you', '<eos>'],
 'fr_tokens': ['<sos>', 'je', 'vous', 'laisse', 'la', 'decision', '<eos>']}

### Dataloaders

In [45]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_en_ids = [example["en_ids"] for example in batch]
        batch_fr_ids = [example["fr_ids"] for example in batch]
        en_lens = [example["en_ids"].shape[0] for example in batch]
        fr_lens = [example["fr_ids"].shape[0] for example in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_fr_ids = nn.utils.rnn.pad_sequence(batch_fr_ids, padding_value=pad_index)
        batch = {
            "en_ids": batch_en_ids,
            "fr_ids": batch_fr_ids,
            "en_lens": en_lens,
            "fr_lens": fr_lens
        }
        return batch

    return collate_fn

In [46]:
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader

In [47]:
batch_size = 128

train_data_loader = get_data_loader(train_dataset, batch_size, pad_index, shuffle=True)
valid_data_loader = get_data_loader(val_dataset, batch_size, pad_index)
test_data_loader = get_data_loader(test_dataset, batch_size, pad_index)

In [48]:
len(valid_data_loader)

107

### Model

In [49]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_lens):
        # src = [src length, batch size]
        embedded = self.dropout(self.embedding(src))
        # embedded = [src length, batch size, embedding dim]
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, np.array(src_lens), enforce_sorted=False, batch_first=False)
        packed_outputs, (hidden, cell) = self.rnn(packed)
        # outputs, (hidden, cell) = self.rnn(embedded)
        # outputs = [src length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # outputs are always from the top hidden layer
        return hidden, cell

In [50]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        # input = [batch size]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # n directions in the decoder will both always be 1, therefore:
        # hidden = [n layers, batch size, hidden dim]
        # context = [n layers, batch size, hidden dim]
        input = input.unsqueeze(0)
        # input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch size, embedding dim]
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output = [seq length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # seq length and n directions will always be 1 in this decoder, therefore:
        # output = [1, batch size, hidden dim]
        # hidden = [n layers, batch size, hidden dim]
        # cell = [n layers, batch size, hidden dim]
        prediction = self.fc_out(output.squeeze(0))
        # prediction = [batch size, output dim]
        return prediction, hidden, cell

In [51]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert (
            encoder.hidden_dim == decoder.hidden_dim
        ), "Hidden dimensions of encoder and decoder must be equal!"
        assert (
            encoder.n_layers == decoder.n_layers
        ), "Encoder and decoder must have equal number of layers!"

    def forward(self, src, src_lens, trg, trg_lens, teacher_forcing_ratio):
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        batch_size = trg.shape[1]
        trg_length = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)
        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src, src_lens)
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # first input to the decoder is the <sos> tokens
        input = trg[0, :]
        # input = [batch size]
        for t in range(1, trg_length):
            # insert input token embedding, previous hidden and previous cell states
            # receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            # output = [batch size, output dim]
            # hidden = [n layers, batch size, hidden dim]
            # cell = [n layers, batch size, hidden dim]
            # place predictions in a tensor holding predictions for each token
            outputs[t] = output
            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            # get the highest predicted token from our predictions
            top1 = output.argmax(1)
            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[t] if teacher_force else top1
            # input = [batch size]
        return outputs

### Training

In [55]:
input_dim = len(fr_vocab)
output_dim = len(en_vocab)
encoder_embedding_dim = 256
decoder_embedding_dim = 256
hidden_dim = 512
n_layers = 2
encoder_dropout = 0.5
decoder_dropout = 0.5
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    hidden_dim,
    n_layers,
    encoder_dropout,
)

decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    hidden_dim,
    n_layers,
    decoder_dropout,
)

model = Seq2Seq(encoder, decoder, device).to(device)

In [56]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)


model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(12220, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(8146, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=8146, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [57]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 16,749,010 trainable parameters


In [58]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

In [60]:
def train_fn(
    model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device
):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(data_loader):
        src = batch["fr_ids"].to(device)
        src_lens = batch["fr_lens"]
        trg = batch["en_ids"].to(device)
        trg_lens = batch["en_lens"]
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        optimizer.zero_grad()
        output = model(src, src_lens, trg, trg_lens, teacher_forcing_ratio)
        # output = [trg length, batch size, trg vocab size]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        # output = [(trg length - 1) * batch size, trg vocab size]
        trg = trg[1:].view(-1)
        # trg = [(trg length - 1) * batch size]
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [61]:
def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            src = batch["fr_ids"].to(device)
            src_lens = batch["fr_lens"]
            trg = batch["en_ids"].to(device)
            trg_lens = batch["en_lens"]
            # src = [src length, batch size]
            # trg = [trg length, batch size]
            output = model(src, src_lens, trg, trg_lens, 0)  # turn off teacher forcing
            # output = [trg length, batch size, trg vocab size]
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            # output = [(trg length - 1) * batch size, trg vocab size]
            trg = trg[1:].view(-1)
            # trg = [(trg length - 1) * batch size]
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [94]:
n_epochs = 10
clip = 1.0
teacher_forcing_ratio = 0.5

best_valid_loss = float("inf")

for epoch in tqdm.tqdm(range(n_epochs)):
    train_loss = train_fn(
        model,
        train_data_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )
    valid_loss = evaluate_fn(
        model,
        valid_data_loader,
        criterion,
        device,
    )
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "tut1-model.pt")
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")

 10%|██████▊                                                             | 1/10 [02:55<26:19, 175.55s/it]

	Train Loss:   4.425 | Train PPL:  83.501
	Valid Loss:   3.916 | Valid PPL:  50.192


 20%|█████████████▌                                                      | 2/10 [06:14<25:12, 189.04s/it]

	Train Loss:   3.336 | Train PPL:  28.098
	Valid Loss:   3.306 | Valid PPL:  27.276


 30%|████████████████████▍                                               | 3/10 [10:02<24:08, 206.88s/it]

...

	Train Loss:   1.685 | Train PPL:   5.395
	Valid Loss:   2.351 | Valid PPL:  10.498


 90%|█████████████████████████████████████████████████████████████▏      | 9/10 [46:44<06:28, 388.62s/it]

	Train Loss:   1.587 | Train PPL:   4.888
	Valid Loss:   2.294 | Valid PPL:   9.919


100%|███████████████████████████████████████████████████████████████████| 10/10 [54:26<00:00, 326.62s/it]

	Train Loss:   1.497 | Train PPL:   4.470
	Valid Loss:   2.262 | Valid PPL:   9.601


### Evaluating on test set

In [62]:
model.load_state_dict(torch.load("tut1-model.pt"))

test_loss = evaluate_fn(model, test_data_loader, criterion, device)

print(f"| Test Loss: {test_loss:.3f} | Test PPL: {np.exp(test_loss):7.3f} |")

| Test Loss: 2.186 | Test PPL:   8.897 |


### BLEU score

In [63]:
def translate_sentence(
    sentence,
    model,
    en_nlp,
    fr_nlp,
    en_vocab,
    fr_vocab,
    lower,
    sos_token,
    eos_token,
    device,
    max_output_length=25,
):
    model.eval()
    with torch.no_grad():
        if isinstance(sentence, str):
            tokens = [token.text for token in fr_nlp.tokenizer(sentence)]
        else:
            tokens = [token for token in sentence]
        if lower:
            tokens = [token.lower() for token in tokens]
        tokens = [sos_token] + tokens + [eos_token]
        ids = fr_vocab.lookup_indices(tokens)
        tensor = torch.LongTensor(ids).unsqueeze(-1).to(device)
        src_len = [tensor.shape[0]]
        hidden, cell = model.encoder(tensor, src_len)
        inputs = en_vocab.lookup_indices([sos_token])
        for _ in range(max_output_length):
            inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
            output, hidden, cell = model.decoder(inputs_tensor, hidden, cell)
            predicted_token = output.argmax(-1).item()
            inputs.append(predicted_token)
            if predicted_token == en_vocab[eos_token]:
                break
        tokens = en_vocab.lookup_tokens(inputs)
    return tokens

In [67]:
sentence = test_data[0]["fr"]
expected_translation = test_data[0]["en"]

sentence, expected_translation

('les dechets d usines polluent quelquefois nos rivieres',
 'factory waste sometimes pollutes our rivers')

In [68]:
translation = translate_sentence(
    sentence,
    model,
    en_nlp,
    fr_nlp,
    en_vocab,
    fr_vocab,
    lower,
    sos_token,
    eos_token,
    device,
)

In [69]:
translation

['<sos>', 'prisons', '<unk>', '<unk>', '<unk>', 'their', '<eos>']

In [70]:
sentence = "Bonjour."

In [71]:
translation = translate_sentence(
    sentence,
    model,
    en_nlp,
    fr_nlp,
    en_vocab,
    fr_vocab,
    lower,
    sos_token,
    eos_token,
    device,
)

In [72]:
translation

['<sos>', 'let', 's', 'spoil', '<eos>']

In [ ]:
translations = [
    translate_sentence(
        example["fr"],
        model,
        en_nlp,
        fr_nlp,
        en_vocab,
        fr_vocab,
        lower,
        sos_token,
        eos_token,
        device,
    )
    for example in tqdm.tqdm(test_data)
]

 37%|███████████████████████▏                                       | 5007/13585 [01:21<02:31, 56.61it/s]

In [4]:
bleu = evaluate.load("bleu")

In [ ]:
predictions = [" ".join(translation[1:-1]) for translation in translations]

references = [[example["en"]] for example in test_data]

In [ ]:
predictions[0], references[0]

In [ ]:
def get_tokenizer_fn(nlp, lower):
    def tokenizer_fn(s):
        tokens = [token.text for token in nlp.tokenizer(s)]
        if lower:
            tokens = [token.lower() for token in tokens]
        return tokens

    return tokenizer_fn

In [ ]:
tokenizer_fn = get_tokenizer_fn(en_nlp, lower)

In [ ]:
tokenizer_fn(predictions[0]), tokenizer_fn(references[0][0])

In [ ]:
results = bleu.compute(
    predictions=predictions, references=references, tokenizer=tokenizer_fn
)

In [ ]:
results